In [1]:
#import statements
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.feature_extraction import text

In [2]:
from nltk.corpus import stopwords
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer

In [3]:
import re
import string

In [4]:
#load and clean contents

df = pd.read_csv('doj.csv')
df = df.drop(columns = ['id', 'topics', 'components'], axis = 1)
df.dropna(subset = ['title', 'date'], inplace=True)

df['contents'] = df['contents'].astype(str)
df['contents_processed'] = \
df['contents'].map(lambda x: re.sub('[,\.!?]','',x))
df['contents_processed'] = \
df['contents_processed'].map(lambda x: x.lower())
df['contents_processed'].head()

df = df.drop(columns = ['contents'], axis = 1)

In [5]:
df

title        date  \
0      District Court Enters Permanent Injunction Aga...  2018-07-27   
1      Justice Department Announces Resolution  With ...  2018-07-27   
2      Kansas City Area Laboratory Owner Convicted of...  2018-07-27   
3      3M Company Agrees to Pay $9.1 Million to Resol...  2018-07-26   
4      Amite Residents Charged with Civil Rights Crim...  2018-07-26   
...                                                  ...         ...   
17819  Jordanian National Arrested in New York to Fac...  2018-07-30   
17820  Former Virginia High School Science Teacher Se...  2018-07-30   
17821  Former Owner of Sleep Study Businesses Convict...  2018-07-30   
17822  Attorney General Sessions Delivers Remarks at ...  2018-07-30   
17823  Kansas City Area Laboratory Owner Convicted of...  2018-07-27   

                                      contents_processed  
0      the us district court for the northern distric...  
1      the department of justice announced today that...  
2      yesterday a federal judge found ahmed el-sheri...  
3      the department of justice announced today that...  
4      a federal grand jury in new orleans louisiana ...  
...                                                  ...  
17819  washington – a jordanian national residing in ...  
17820  a charlottesville virginia man was sentenced t...  
17821  a federal jury convicted a sterling virginia w...  
17822  remarks as prepared for delivery\r\nthank you ...  
17823  yesterday a federal judge found ahmed el-sheri...  

[17759 rows x 3 columns]

In [6]:
# Way around missing packages that for some reason I can't download the normal way

import os
import nltk

# Create NLTK data directory
NLTK_DATA_DIR = './nltk_data'
if not os.path.exists(NLTK_DATA_DIR):
        os.makedirs(NLTK_DATA_DIR)

nltk.data.path.append(NLTK_DATA_DIR)

# Download packages and store in directory above
nltk.download('punkt', download_dir=NLTK_DATA_DIR)
nltk.download('averaged_perceptron_tagger', download_dir=NLTK_DATA_DIR)
nltk.download('wordnet', download_dir=NLTK_DATA_DIR)

[nltk_data] Downloading package punkt to ./nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     ./nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to ./nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
def nouns(text):
    is_noun = lambda pos : pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    wordnet_lemmatizer = WordNetLemmatizer()
    all_nouns = [wordnet_lemmatizer.lemmatize(word) \
                for (word, pos) in pos_tag(tokenized) if is_noun(pos)]
    return ' '.join(all_nouns)

data_nouns = pd.DataFrame(df.contents_processed.apply(nouns))

data_nouns.head()

contents_processed
0  district court district illinois injunction ch...
1  department justice today mirelis sa resolution...
2  yesterday judge owner operator radiation safet...
3  department justice today company st paul minne...
4  jury orleans trafficking hate crime indictment...

In [8]:
#removing extra stop words
stop_noun = ['america', 'today', 'thing', 'wwwjusticegov']
stop_words_noun_agg = text.ENGLISH_STOP_WORDS.union(stop_noun)

tv_noun = TfidfVectorizer(stop_words=stop_words_noun_agg, ngram_range = (1,1), max_df = 0.9, min_df = 0.1)

data_tv_noun = tv_noun.fit_transform(data_nouns.contents_processed)

data_dtm_noun = pd.DataFrame(data_tv_noun.toarray(), columns = tv_noun.get_feature_names())

data_dtm_noun.index = df.index

data_dtm_noun.head()

/opt/homebrew/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


abuse  access   account       act    action  activity  addition  \
0  0.000000     0.0  0.000000  0.046884  0.111119  0.000000  0.000000   
1  0.000000     0.0  0.875105  0.000000  0.000000  0.037366  0.020830   
2  0.000000     0.0  0.000000  0.217431  0.085888  0.000000  0.000000   
3  0.186685     0.0  0.000000  0.113265  0.067112  0.000000  0.000000   
4  0.102208     0.0  0.000000  0.124022  0.000000  0.000000  0.071102   

   administration    agency     agent  ...       use  victim  violation  \
0             0.0  0.000000  0.000000  ...  0.067917     0.0   0.059651   
1             0.0  0.000000  0.000000  ...  0.000000     0.0   0.000000   
2             0.0  0.081954  0.131507  ...  0.000000     0.0   0.184426   
3             0.0  0.064037  0.102758  ...  0.082039     0.0   0.000000   
4             0.0  0.000000  0.000000  ...  0.000000     0.0   0.552278   

      visit  washington   website  wire      work      year  york  
0  0.073853    0.000000  0.061856   0.0  0.000000  0.000000   0.0  
1  0.000000    0.000000  0.011984   0.0  0.000000  0.006874   0.0  
2  0.000000    0.082538  0.000000   0.0  0.097056  0.109689   0.0  
3  0.000000    0.000000  0.000000   0.0  0.000000  0.000000   0.0  
4  0.000000    0.000000  0.000000   0.0  0.000000  0.140774   0.0  

[5 rows x 181 columns]

In [9]:
list(data_dtm_noun.columns)

['abuse',
 'access',
 'account',
 'act',
 'action',
 'activity',
 'addition',
 'administration',
 'agency',
 'agent',
 'agreement',
 'allegation',
 'american',
 'announcement',
 'april',
 'area',
 'assistance',
 'assistant',
 'authority',
 'bank',
 'behalf',
 'benefit',
 'bureau',
 'business',
 'california',
 'care',
 'case',
 'center',
 'charge',
 'chief',
 'citizen',
 'city',
 'claim',
 'commitment',
 'community',
 'company',
 'complaint',
 'compliance',
 'conduct',
 'connection',
 'conspiracy',
 'conspirators',
 'control',
 'cooperation',
 'corporation',
 'counsel',
 'count',
 'country',
 'county',
 'court',
 'crime',
 'customer',
 'date',
 'day',
 'defendant',
 'deputy',
 'director',
 'district',
 'division',
 'document',
 'dollar',
 'drug',
 'effort',
 'employee',
 'enforcement',
 'evidence',
 'exchange',
 'executive',
 'facility',
 'fact',
 'family',
 'fbi',
 'field',
 'fine',
 'florida',
 'force',
 'form',
 'fraud',
 'fund',
 'general',
 'government',
 'group',
 'health',
 'home

In [10]:
#iterate through topics
def display_topics(model, feature_names, num_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic", ix)
        else:
            print("\nTopic: '", topic_names[ix], "'")
        print(",".join([feature_names[i] \
            for i in topic.argsort()[:-num_top_words - 1:-1]]))

In [11]:
nmf_model = NMF(10)

doc_topic = nmf_model.fit_transform(data_dtm_noun)

display_topics(nmf_model, tv_noun.get_feature_names(), 10)

/opt/homebrew/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(



Topic 0
district,prison,case,division,trial,office,court,judge,charge,plea

Topic 1
tax,return,irs,income,business,revenue,customer,division,deputy,year

Topic 2
health,care,fraud,claim,service,program,home,provider,office,force

Topic 3
settlement,company,state,agreement,act,claim,government,lawsuit,complaint,compliance

Topic 4
right,officer,division,county,complaint,act,law,lawsuit,person,district

Topic 5
law,enforcement,community,crime,state,officer,agency,task,program,force

Topic 6
indictment,defendant,count,charge,conspiracy,jury,year,prison,fine,fbi

Topic 7
fraud,company,bank,scheme,account,money,conspiracy,wire,payment,business

Topic 8
victim,crime,defendant,abuse,family,home,case,initiative,individual,prosecution

Topic 9
drug,complaint,defendant,administration,member,organization,enforcement,operation,office,district


/opt/homebrew/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [30]:
#setting max rows to see more throughout process

pd.set_option('display.max_rows', 500)

In [13]:
#splitting documents by presidency

obama = df.loc[df['date'].str.contains('2013|2014|2015|2016')]
trump = df.loc[df['date'].str.contains('2017|2018|2019|2020')]

In [14]:
# obama vectorization

obama_nouns = pd.DataFrame(obama.contents_processed.apply(nouns))

obama_tv_noun = tv_noun.fit_transform(obama_nouns.contents_processed)

obama_dtm_noun = pd.DataFrame(obama_tv_noun.toarray(), columns = tv_noun.get_feature_names())

obama_dtm_noun.index = obama.index

obama_dtm_noun.head()

access   account       act    action  activity  addition  \
2208     0.0  0.000000  0.000000  0.000000  0.000000  0.000000   
2209     0.0  0.000000  0.035346  0.081531  0.000000  0.038378   
2210     0.0  0.248852  0.036102  0.000000  0.073647  0.019599   
2211     0.0  0.000000  0.238984  0.000000  0.054169  0.000000   
2212     0.0  0.000000  0.000000  0.000000  0.000000  0.000000   

      administration    agency     agent  agreement  ...       use    victim  \
2208        0.000000  0.000000  0.053586   0.000000  ...  0.000000  0.000000   
2209        0.000000  0.000000  0.185083   0.154334  ...  0.000000  0.000000   
2210        0.026409  0.000000  0.000000   0.059113  ...  0.025294  0.000000   
2211        0.000000  0.044411  0.000000   0.000000  ...  0.000000  0.048722   
2212        0.000000  0.000000  0.065646   0.082110  ...  0.000000  0.000000   

      violation     visit  washington   website      wire      work      year  \
2208    0.00000  0.000000    0.000000  0.075679  0.000000  0.000000  0.093784   
2209    0.04427  0.000000    0.053239  0.000000  0.056017  0.000000  0.000000   
2210    0.00000  0.000000    0.000000  0.022248  0.000000  0.023981  0.000000   
2211    0.00000  0.059589    0.000000  0.049092  0.000000  0.000000  0.000000   
2212    0.00000  0.000000    0.000000  0.092712  0.119210  0.000000  0.057446   

      york  
2208   0.0  
2209   0.0  
2210   0.0  
2211   0.0  
2212   0.0  

[5 rows x 180 columns]

In [15]:
nmf_model = NMF(5)

obama_doc_topic = nmf_model.fit_transform(obama_dtm_noun)

display_topics(nmf_model, tv_noun.get_feature_names(), 10)

/opt/homebrew/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(



Topic 0
district,indictment,charge,conspiracy,defendant,count,division,case,office,fraud

Topic 1
tax,return,income,irs,business,revenue,account,year,bank,customer

Topic 2
health,care,fraud,claim,service,home,program,office,provider,center

Topic 3
settlement,company,state,agreement,act,government,claim,complaint,lawsuit,program

Topic 4
right,officer,law,police,community,county,victim,enforcement,division,district


/opt/homebrew/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [16]:
# trump vectorization

trump_nouns = pd.DataFrame(trump.contents_processed.apply(nouns))

trump_tv_noun = tv_noun.fit_transform(trump_nouns.contents_processed)

trump_dtm_noun = pd.DataFrame(trump_tv_noun.toarray(), columns = tv_noun.get_feature_names())

trump_dtm_noun.index = trump.index

trump_dtm_noun.head()

Bad pipe message: %s [b'\x88~\xd1C\xb0\xbc\xe0#\x91\x06\xc7\x1fJ\xd0\x19(\x0b~ H\xa5"\xecK\xc0\x18\x89\xb1E\xe5\xbfB\xe7\xbdTp\x02\xe4\xfb,\xf4!\x1c\xa4$\x90\xd7\xca\xc0\\1\x00 jj\x13\x01\x13\x02\x13\x03\xc0+\xc0/\xc0,\xc00\xcc\xa9\xcc\xa8\xc0\x13\xc0\x14\x00\x9c\x00\x9d\x00/\x005\x01\x00\x01\x93JJ\x00\x00\x00\x17\x00\x00\xff\x01\x00\x01\x00\x00\n\x00\n\x00\x08jj\x00\x1d\x00\x17\x00\x18\x00\x0b\x00\x02\x01\x00\x00#\x00\x00\x00\x10\x00\x0e\x00\x0c\x02h2\x08http/1.1\x00\x05\x00\x05\x01\x00\x00\x00\x00\x00\r\x00\x14\x00\x12\x04\x03\x08\x04\x04\x01\x05\x03\x08\x05\x05\x01\x08\x06\x06\x01\x02\x01\x00\x12\x00\x00\x003\x00', b')jj\x00\x01\x00\x00\x1d\x00 \x1cMi\t\x9e\xb9)4\x13(r\xce\xf7\xd6\xf9\xbb\xb1Ry\xab\xb8\xdaR4\xde\xa6\xaf\xbc\xeey\x9d\x01']


abuse  access   account       act    action  activity  addition  \
0  0.000000     0.0  0.000000  0.047643  0.112072  0.000000  0.000000   
1  0.000000     0.0  0.869922  0.000000  0.000000  0.037663  0.021680   
2  0.000000     0.0  0.000000  0.221367  0.086787  0.000000  0.000000   
3  0.165542     0.0  0.000000  0.113144  0.066537  0.000000  0.000000   
4  0.090695     0.0  0.000000  0.123976  0.000000  0.000000  0.071243   

   administration    affair    agency  ...     visit  washington  way  \
0             0.0  0.074045  0.000000  ...  0.074901    0.000000  0.0   
1             0.0  0.000000  0.000000  ...  0.000000    0.000000  0.0   
2             0.0  0.000000  0.077542  ...  0.000000    0.106764  0.0   
3             0.0  0.000000  0.059449  ...  0.000000    0.000000  0.0   
4             0.0  0.000000  0.000000  ...  0.000000    0.000000  0.0   

    website  william  wire     woman      work      year  york  
0  0.056004      0.0   0.0  0.000000  0.000000  0.000000   0.0  
1  0.011087      0.0   0.0  0.000000  0.000000  0.007033   0.0  
2  0.000000      0.0   0.0  0.000000  0.089854  0.110043   0.0  
3  0.000000      0.0   0.0  0.178048  0.000000  0.000000   0.0  
4  0.000000      0.0   0.0  0.097547  0.000000  0.138666   0.0  

[5 rows x 196 columns]

In [17]:
nmf_model = NMF(5)

obama_doc_topic = nmf_model.fit_transform(trump_dtm_noun)

display_topics(nmf_model, tv_noun.get_feature_names(), 10)

/opt/homebrew/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(



Topic 0
indictment,district,charge,count,defendant,division,conspiracy,fraud,office,security

Topic 1
tax,return,irs,income,deputy,division,business,assistant,revenue,prison

Topic 2
health,fraud,care,claim,program,service,drug,force,inspector,scheme

Topic 3
settlement,right,company,state,agreement,complaint,act,division,claim,government

Topic 4
law,crime,enforcement,officer,drug,state,victim,police,community,right


/opt/homebrew/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [27]:
#looking at vectorizor in order to see what words might need to be added to the stop word list
text = " ".join(review for review in df.contents_processed)

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words = 'english')
matrix = vectorizer.fit_transform([text])
matrix

<1x80026 sparse matrix of type '<class 'numpy.int64'>'
	with 80026 stored elements in Compressed Sparse Row format>

In [33]:
counts = pd.DataFrame(matrix.toarray(),
                      columns=vectorizer.get_feature_names())
counts.T.sort_values(by=0, ascending=False).head(200)

/opt/homebrew/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


0
department      64917
attorney        62947
district        46911
justice         46623
division        45144
general         35006
criminal        33014
office          32134
assistant       31394
federal         30603
fraud           29649
tax             27952
today           27319
enforcement     26317
case            26048
court           24575
states          24558
law             23950
united          23723
said            22941
guilty          21053
civil           20534
information     19115
years           18990
fbi             18751
investigation   18336
prison          17370
trial           17329
new             16797
health          16741
million         15867
according       15745
services        15555
indictment      15430
rights          15405
including       15354
defendants      15331
conspiracy      15315
government      15193
act             15192
false           14860
state           14256
special         14118
announced       14041
service         13494
charge          13362
section         12730
pleaded         12673
public          12381
national        12336
medicare        12273
count           12122
irs             11997
pay             11878
company         11472
care            11123
sentenced       10907
settlement      10642
force           10623
individuals     10518
acting          10345
financial       10107
security        10039
charged         10035
work             9943
filed            9899
agent            9862
agreement        9832
scheme           9795
judge            9764
program          9720
complaint        9671
victims          9476
claims           9463
efforts          9274
sentence         9259
used             9221
crime            9198
charges          9150
documents        9034
police           8989
attorneys        8692
admitted         8687
business         8540
based            8539
washington       8419
assistance       8308
money            8252
year             8206
bank             8107
members          8094
provided         8046
eastern          7953
prosecuted       7889
months           7838
returns          7817
drug             7782
use              7762
approximately    7758
crimes           7574
york             7563
companies        7553
field            7449
investigated     7419
fraudulent       7414
antitrust        7276
maximum          7271
county           7254
related          7121
southern         7064
plea             7059
american         6926
provide          6918
international    6906
deputy           6760
cases            6746
defendant        6728
order            6695
continue         6658
alleged          6646
income           6605
texas            6495
conduct          6493
2009             6472
evidence         6413
agreed           6381
addition         6325
local            6313
florida          6295
alleges          6295
foreign          6288
support          6269
people           6264
ensure           6094
inspector        6040
time             5948
country          5925
sentencing       5872
john             5841
agents           5833
accounts         5826
counts           5788
commit           5780
programs         5770
home             5767
conspirators     5753
task             5734
release          5687
partners         5608
fine             5597
resources        5570
chief            5546
working          5540
additional       5508
medical          5495
director         5482
housing          5469
laws             5466
supervised       5456
california       5441
convicted        5439
agencies         5414
discrimination   5412
city             5411
officers         5330
human            5329
community        5266
hhs              5230
10               5221
protect          5185
illegal          5181
allegedly        5152
jury             5103
allegations      5073
committed        5070
serve            5057
child            5020
investigations   5006
trafficking      4956
2011             4938
result           4935
march           